# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [14]:
class State:
    def __init__(self, husbands, wives, boat):
        self.husbands = husbands.copy()  
        self.wives = wives.copy()        
        self.boat = boat  #(0=left, 1=island, 2=right)
    
    def __eq__(self, other):
        return (self.husbands == other.husbands and 
                self.wives == other.wives and 
                self.boat == other.boat)
    
    def __hash__(self):
        return hash((tuple(self.husbands), tuple(self.wives), self.boat))

    def __lt__(self, other):
        return self.boat < other.boat
    
    def __le__(self, other):
        return self.boat <= other.boat
    
    def __gt__(self, other):
        return self.boat > other.boat
    
    
    def is_valid(self):
        for location in [0, 1, 2]: 
            wives_here = [i for i, loc in enumerate(self.wives) if loc == location]
            if not wives_here:
                continue
            
            husbands_here = [i for i, loc in enumerate(self.husbands) if loc == location]
            
            for wife_idx in wives_here:
                if self.husbands[wife_idx] != location:
                    if husbands_here:
                        return False
        
        return True
    
    def is_goal(self, n):
        return all(h == 2 for h in self.husbands) and all(w == 2 for w in self.wives)
    
    def __str__(self):
        locations = ["Left", "Island", "Right"]
        boat_loc = locations[self.boat]
        
        result = f"Boat: {boat_loc}\n"
        for i in range(len(self.husbands)):
            result += f"Couple {i}: Husband at {locations[self.husbands[i]]}, Wife at {locations[self.wives[i]]}\n"
        return result

In [12]:
def get_successors(state, n):
    successors = []
    
    people_at_boat = []
    for i in range(n):
        if state.husbands[i] == state.boat:
            people_at_boat.append(('H', i))
        if state.wives[i] == state.boat:
            people_at_boat.append(('W', i))
    
    if state.boat == 0: 
        destinations = [1]  
    elif state.boat == 1:  
        destinations = [0, 2] 
    else: 
        destinations = [1] 
    
    for destination in destinations:
        for person in people_at_boat:
            new_state = move_people(state, [person], destination)
            if new_state.is_valid():
                successors.append(new_state)
        
        for i in range(len(people_at_boat)):
            for j in range(i+1, len(people_at_boat)):
                new_state = move_people(state, [people_at_boat[i], people_at_boat[j]], destination)
                if new_state.is_valid():
                    successors.append(new_state)
    
    return successors

def move_people(state, people, destination):
    new_husbands = state.husbands.copy()
    new_wives = state.wives.copy()
    
    for person_type, idx in people:
        if person_type == 'H':
            new_husbands[idx] = destination
        else: 
            new_wives[idx] = destination
    
    return State(new_husbands, new_wives, destination)

In [9]:
from collections import deque
import heapq


In [26]:
def bkt_solve(n):
    initial_state = State([0] * n, [0] * n, 0)
    
    visited_on_current_path = set()

    def bkt_recursive_helper(current_state, path_history):
        if current_state.is_goal(n):
            return path_history + [current_state]

        visited_on_current_path.add(current_state)

        for successor in get_successors(current_state, n):
            if successor not in visited_on_current_path: 
                result = bkt_recursive_helper(successor, path_history + [current_state])
                
                if result:
                    return result
                
        if current_state in visited_on_current_path: 
            visited_on_current_path.remove(current_state)
        return None

    solution_path = bkt_recursive_helper(initial_state, [])
    
    return solution_path

In [34]:
def dfs_solve(n, track_visited=True):
    initial_state = State([0] * n, [0] * n, 0)
    visited = set()
    
    def dfs(state, path):
        if state.is_goal(n):
            return path + [state]
        
        visited.add(state)
        
        successors = get_successors(state, n)
        
        def priority_score(s):
            right_bank_count = sum(1 for h in s.husbands if h == 2) + sum(1 for w in s.wives if w == 2)
            return right_bank_count
        
        successors.sort(key=priority_score, reverse=True)
        
        for successor in successors:
            if successor not in visited:
                result = dfs(successor, path + [state])
                if result:
                    return result
        
        if not track_visited:
            visited.remove(state) 
        
        return None
    
    return dfs(initial_state, [])

In [8]:
def a_star_solve(n):
    
    initial_state = State([0] * n, [0] * n, 0)
    
    def heuristic(state):
        return sum(1 for h in state.husbands if h != 2) + sum(1 for w in state.wives if w != 2)
    
    queue = [(heuristic(initial_state), 0, initial_state, [])]
    visited = {initial_state: 0}  
    while queue:
        _, g, state, path = heapq.heappop(queue)
        
        if state.is_goal(n):
            return path + [state]
        
        for successor in get_successors(state, n):
            new_g = g + 1
            
            if successor not in visited or new_g < visited[successor]:
                visited[successor] = new_g
                f = new_g + heuristic(successor)
                heapq.heappush(queue, (f, new_g, successor, path + [state]))
    
    return None

In [27]:
def print_solution(path, algorithm_name):
    if not path:
        print(f"No solution found using {algorithm_name}!")
        return
    
    print(f"\n{algorithm_name} solution found in {len(path)-1} moves:")
    
    print("\nInitial state:")
    print(path[0])
    
    for i in range(1, len(path)):
        print(f"\nMove {i}:")
        
        prev, curr = path[i-1], path[i]
        moved_people = []
        
        for j in range(len(prev.husbands)):
            if prev.husbands[j] != curr.husbands[j]:
                moved_people.append(f"Husband {j}")
            if prev.wives[j] != curr.wives[j]:
                moved_people.append(f"Wife {j}")
        
        direction = "→" if prev.boat < curr.boat else "←"
        locations = ["Left", "Island", "Right"]
        print(f"  {', '.join(moved_people)} moved from {locations[prev.boat]} {direction} {locations[curr.boat]}")
        print(curr)

def solve_jealous_husbands(n=3):
    print("BKT:")    
    bkt_solution = bkt_solve(n)
    print_solution(bkt_solution, "BKT")
    
    print("\nDFS")
    dfs_solution = dfs_solve(n)
    print_solution(dfs_solution, "DFS")
    
    print("\nA*")
    astar_solution = a_star_solve(n)
    print_solution(astar_solution, "A*")

In [35]:
solve_jealous_husbands(3)

BKT:

BKT solution found in 206 moves:

Initial state:
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 1:
  Husband 0, Wife 0 moved from Left → Island
Boat: Island
Couple 0: Husband at Island, Wife at Island
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 2:
  Husband 0 moved from Island ← Left
Boat: Left
Couple 0: Husband at Left, Wife at Island
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 3:
  Wife 1 moved from Left → Island
Boat: Island
Couple 0: Husband at Left, Wife at Island
Couple 1: Husband at Left, Wife at Island
Couple 2: Husband at Left, Wife at Left


Move 4:
  Wife 0 moved from Island ← Left
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Island
Couple 2: Husband at Left, Wife at Left


Move 5:
  Husband 1 moved from Left → Island
Boat: Island
Couple 0: Husband at Lef